In [2]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import cv2
from PIL import Image
import PIL
from google.colab.patches import cv2_imshow
import gspread
import torch
import torchvision
import torchvision.transforms as transforms
import sklearn
from skimage.transform import resize
from skimage.io import imread

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
NoFind = 0
Covid = 1
Atelectasis = 3
Cardiomegaly = 4
Effusion = 5
Infiltration = 6
MASS = 7
Nodule = 8
Pneumonia = 9
Pneumothorax = 10
Consolidation = 11
Edema = 12
Emphysema, Fibrosis, Pleural, Hernia, Tuberculosis, Opacity, Other = 13, 14, 15, 16, 17, 18, 19
testing_path = '/content/gdrive/MyDrive/DiseaseDetection/Testing/Images/'

tb_path_pos = '/content/gdrive/MyDrive/DiseaseDetection/TB_Chest_Radiography_Database/Tuberculosis/'
tb_path_neg = '/content/gdrive/MyDrive/DiseaseDetection/TB_Chest_Radiography_Database/Normal/'
tb_data = '/content/gdrive/MyDrive/DiseaseDetection/TB_Chest_Radiography_Database/Tuberculosis.csv'

cov1_path_cov = '/content/gdrive/MyDrive/DiseaseDetection/COVID-19_Radiography_Dataset/COVID/images/'
cov1_path_vir = '/content/gdrive/MyDrive/DiseaseDetection/COVID-19_Radiography_Dataset/Viral Pneumonia/images/'
cov1_path_opac = '/content/gdrive/MyDrive/DiseaseDetection/COVID-19_Radiography_Dataset/Lung_Opacity/images/'
cov1_path_neg = '/content/gdrive/MyDrive/DiseaseDetection/COVID-19_Radiography_Dataset/Normal/images/'
cov1_data = '/content/gdrive/MyDrive/DiseaseDetection/COVID-19_Radiography_Dataset/data.csv'

cov2_images = '/content/gdrive/MyDrive/DiseaseDetection/covid-chestxray-dataset-master/images/'
cov2_data = '/content/gdrive/MyDrive/DiseaseDetection/covid-chestxray-dataset-master/Data.csv'

dd_paths = []
for i in range(1, 13):
  x = '/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/images_0'
  if i < 10:
    x += '0'
  x += str(i) + '/images/'
  dd_paths.append(x)

dd_data = '/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/data.csv'

In [4]:
tb_df = pd.read_csv(tb_data)
cov1_df = pd.read_csv(cov1_data)
cov2_df = pd.read_csv(cov2_data)
dd_df = pd.read_csv(dd_data)

tb_df.rename(columns = {"FILE NAME": "Name"}, inplace = True)
cov1_df.rename(columns = {"FILE NAME": "Name"}, inplace = True)
cov2_df.rename(columns = {"filename": "Name", 'finding': 'Diagnosis'}, inplace = True)
dd_df.rename(columns = {"Image Index": "Name", 'Finding Labels': 'Diagnosis'}, inplace = True)

def split(x):
  if "|" in x:
    return x.split('|')
  elif "/" in x:
    return x.split('/')
  else:
    if type(x) != list:
      return [x]
    else:
      return x
def organize(x):
  for i in range(len(x)):
    if x[i].lower() == "covid" or x[i].lower() == "covid-19":
      x[i] = Covid
    elif x[i] == "None" or x[i] == "No Finding":
      x[i] = NoFind
    elif x[i] == "Viral":
      x[i] = Pneumonia
    elif x[i] == "Atelectasis":
      x[i] = Atelectasis
    elif x[i] == "Cardiomegaly":
      x[i] = Cardiomegaly
    elif x[i] == "Effusion":
      x[i] = Effusion
    elif x[i] == "Infiltration":
      x[i] = Infiltration
    elif x[i] == "Mass":
      x[i] = MASS
    elif x[i] == "Nodule":
      x[i] = Nodule
    elif x[i] == "Pneumonia":
      x[i] = Pneumonia
    elif x[i] == "Pneumothorax":
      x[i] = Pneumothorax
    elif x[i] == "Consolidation":
      x[i] = Consolidation
    elif x[i] == "Edema":
      x[i] = Edema
    elif x[i] == "Emphysema":
      x[i] = Emphysema
    elif x[i] == "Fibrosis":
      x[i] = Fibrosis
    elif x[i] == "Pleural_Thickening":
      x[i] = Pleural
    elif x[i] == "Hernia":
      x[i] = Hernia
    elif x[i] == "Tuberculosis":
      x[i] = Tuberculosis
    elif x[i] == "Opacity":
      x[i] = Opacity
    else:
      x[i] = Other
  return x
def cov1_modify(x):
  if "NORMAL" in x:
    return x.capitalize() + '.png'
  return x + '.png'

def tb_modify(x):
  if "Normal" in x:
    return "Normal TB-" + x.split("-")[1] + '.png'
  return x + ".png"
cov1_df["Name"] = cov1_df["Name"].apply(cov1_modify)
tb_df["Name"] = tb_df["Name"].apply(tb_modify)


df = pd.concat([dd_df, cov2_df, cov1_df, tb_df], ignore_index=True)
df["Diagnosis"] = df["Diagnosis"].apply(split)
df["Diagnosis"] = df["Diagnosis"].apply(organize)
df.rename(columns = {"Name": "Image Name", 'Diagnosis': 'Classification'}, inplace = True)

In [24]:
temp = df["Classification"].copy()
labels = {df["Image Name"][i]: temp[i][0] for i in range(len(df))}
X_test = []
labels_ref = []

In [25]:
for i in os.listdir(tb_path_pos): 
  curr = Image.open(tb_path_pos + i)
  curr = curr.resize((32,32))
  curr = curr.convert("L")
  if i in labels:
    X_test.append(np.asarray(curr).flatten())
    labels_ref.append(labels[i])

In [26]:
for i in os.listdir(tb_path_neg):
  curr = Image.open(tb_path_neg + i)
  curr = curr.resize((32,32))
  curr = curr.convert("L")
  if i in labels:
    X_test.append(np.asarray(curr).flatten())
    labels_ref.append(labels[i])

In [27]:
for i in os.listdir(cov1_path_neg):
  if "(" not in i:
      curr = Image.open(cov1_path_neg + i)
      curr = curr.resize((32,32))
      curr = curr.convert("L")
      if i in labels:
        X_test.append(np.asarray(curr).flatten())
        labels_ref.append(labels[i])

In [28]:
for i in os.listdir(cov1_path_opac):
  curr = Image.open(cov1_path_opac + i)
  curr = curr.resize((32,32))
  curr = curr.convert("L")
  if i in labels:
    X_test.append(np.asarray(curr).flatten())
    labels_ref.append(labels[i])

In [29]:
for i in os.listdir(cov1_path_vir):
  curr = Image.open(cov1_path_vir + i)
  curr = curr.resize((32,32))
  curr = curr.convert("L")
  if i in labels:
    X_test.append(np.asarray(curr).flatten())
    labels_ref.append(labels[i])

In [30]:
for i in os.listdir(cov1_path_cov):
  if "(" not in i:
    curr = Image.open(cov1_path_cov + i)
    curr = curr.resize((32,32))
    curr = curr.convert("L")
    if i in labels:
      X_test.append(np.asarray(curr).flatten())
      labels_ref.append(labels[i])

In [ ]:
import time
start = time.time()
for i in dd_paths:
  print(i, time.time() - start)
  for j in os.listdir(i):
    curr = Image.open(i + j)
    curr = curr.resize((32,32))
    curr = curr.convert("L")
    if j in labels:
      X_test.append(np.asarray(curr).flatten())
      labels_ref.append(labels[j])
end = time.time()
print(end - start)

/content/gdrive/MyDrive/DiseaseDetection/DD Full Dataset/images_001/images/ 0.00018072128295898438


In [16]:
for i in os.listdir(cov2_images):
  curr = Image.open(cov2_images + i)
  curr = curr.resize((32,32))
  curr = curr.convert("L")
  if i in labels:
    X_test.append(np.asarray(curr).flatten())
    labels_ref.append(labels[i])

In [17]:
X_test=np.array(X_test)
target=np.array(labels_ref)

new_df=pd.DataFrame(X_test)
new_df['Target']=target
new_df = new_df[new_df["Target"] != 19]
x=new_df.iloc[:,:-1]
y=new_df.iloc[:,-1]

In [20]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x = scaler.fit_transform(x)

param_grid={}
svc=svm.SVC(probability=True)
model=svm.SVC(kernel='rbf')

In [21]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.40,stratify=y)
print('Splitted Successfully')
model.fit(x_train,y_train)
print('The Model is trained well with the given images')

Splitted Successfully
The Model is trained well with the given images


In [22]:
y_pred=model.predict(x_test)
print("The predicted Data is :")
print(y_pred)
print("The actual data is:")
print(np.array(y_test))
print(f"The model is {sklearn.metrics.accuracy_score(y_pred,y_test)*100}% accurate")

The predicted Data is :
[0 0 9 ... 9 0 1]
The actual data is:
[ 0  0  9 ...  9  0 18]
The model is 80.09347577260587% accurate
